# Attempt with sklearn's MLPClassifier

In [3]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler  
from sklearn.decomposition import PCA
import os

In [4]:
my_path = str(os.getcwd())
X_train = np.load(my_path+'/.data/X_train_surge_new.npz')
Y_train = pd.read_csv(my_path+'/.data/Y_train_surge.csv')
X_test = np.load(my_path+'/.data/X_test_surge_new.npz')

In [5]:
def surge_prediction_metric(dataframe_y_true, dataframe_y_pred):
    weights = np.linspace(1, 0.1, 10)[np.newaxis]
    surge1_columns = [
        'surge1_t0', 'surge1_t1', 'surge1_t2', 'surge1_t3', 'surge1_t4',
        'surge1_t5', 'surge1_t6', 'surge1_t7', 'surge1_t8', 'surge1_t9' ]
    surge2_columns = [
        'surge2_t0', 'surge2_t1', 'surge2_t2', 'surge2_t3', 'surge2_t4',
        'surge2_t5', 'surge2_t6', 'surge2_t7', 'surge2_t8', 'surge2_t9' ]
    surge1_score = (weights * (dataframe_y_true[surge1_columns].values - dataframe_y_pred[surge1_columns].values)**2).mean()
    surge2_score = (weights * (dataframe_y_true[surge2_columns].values - dataframe_y_pred[surge2_columns].values)**2).mean()

    return surge1_score + surge2_score

### Utilitary functions for ruling 'weights' issue

In [6]:
# should be applied to Y_train before learning
def transform(y):
    weights = np.sqrt(np.linspace(1, 0.1, 10)[np.newaxis])
    return weights*y

# should be applied to Y_pred after test
def inverse_transform(y):
    weights = 1/np.sqrt(np.linspace(1, 0.1, 10)[np.newaxis])
    return weights*y

## Use MLP only on surge levels

**Goal**: Is part surges data enough to predict the tide ?

#### 1. First city:

In [86]:
# Extract 1st city data
surge1_y_train = transform(np.array(Y_train)[:,1:11])

surge1_input = np.array(X_train['surge1_input'])
surge1_x_train = surge1_input

surge1_x_test = np.array(X_test['surge1_input'])

In [74]:
clf = MLPRegressor(solver='sgd', hidden_layer_sizes=(10,), alpha=1e-5, random_state=1)
clf.fit(surge1_x_train, surge1_y_train)
surge1_y_test = clf.predict(surge1_x_test)

#### 2. Second city:

In [87]:
# Extract 2nd city data
surge2_y_train = transform(np.array(Y_train)[:,11:])

surge2_input = np.array(X_train['surge2_input'])
surge2_x_train = surge1_input

surge2_x_test = np.array(X_test['surge2_input'])

In [75]:
clf = MLPRegressor(solver='sgd', hidden_layer_sizes=(10,), alpha=1e-5, random_state=1)
clf.fit(surge2_x_train, surge2_y_train)
surge2_y_test = clf.predict(surge2_x_test)

Result of this method: 0.88

Pretty bad, it shows that we need the information of the slp

## Take into account slp

### 1. Dimension reduction of slp data (using PCA)

In [7]:
def slp_to_flat_images(slp, scaler):
    l = []
    for x in slp:
        for i in x:
            l.append(scaler.transform(i.flatten()))
    return np.array(l)

def transform_pca_slp(pca,slp, scaler):
    flat = slp_to_flat_images(slp, scaler)
    slp_LD = pca.transform(flat)
    slp_LD_reshape = np.array([np.concatenate(slp_LD[i*40:(i+1)*40]) for i in range(len(slp))])
    print(slp_LD.shape)
    print(slp_LD_reshape.shape)
    return slp_LD

def fit_transform_pca_slp(slp,nb_comp):
    scaler = StandardScaler()
    s = np.shape(slp)
    scaler.fit(slp.reshape((s[0],s[1]*s[2]*s[3])))
    #flatten
    list_flat_images = slp_to_flat_images(slp, scaler)
    #fit PCA
    pca = PCA(n_components=nb_comp)
    pca.fit(list_flat_images)
    #transform
    slp_LD = transform_pca_slp(pca,slp)
    
    return pca,slp_LD,scaler

In [8]:
pca, slp_train_LD, scaler = fit_transform_pca_slp(X_train['slp'], 4)

TypeError: cannot unpack non-iterable NoneType object

In [3]:
slp_test_LD = transform_pca_slp(pca, X_test['slp'], scaler)

NameError: name 'pca' is not defined

### 2. Prediction

#### 1. First city:

In [86]:
# Extract 1st city data
surge1_y_train = transform(np.array(Y_train)[:,1:11])

surge1_input = np.array(X_train['surge1_input'])
surge1_x_train = np.concatenate((surge1_input, slp_train_LD), axis=1)

surge1_x_test = np.concatenate((np.array(X_test['surge1_input']), slp_test_LD), axis=1)

In [74]:
clf = MLPRegressor(solver='sgd', hidden_layer_sizes=(10,), alpha=1e-5, random_state=1)
clf.fit(surge1_x_train, surge1_y_train)
surge1_y_test = clf.predict(surge1_x_test)

#### 2. Second city:

In [87]:
# Extract 2nd city data
surge2_y_train = transform(np.array(Y_train)[:,11:])

surge2_input = np.array(X_train['surge2_input'])
surge2_x_train = np.concatenate((surge2_input, slp_train_LD), axis=1)

surge2_x_test = np.concatenate((np.array(X_test['surge2_input']), slp_test_LD), axis=1)

In [75]:
clf = MLPRegressor(solver='sgd', hidden_layer_sizes=(10,), alpha=1e-5, random_state=1)
clf.fit(surge2_x_train, surge2_y_train)
surge2_y_test = clf.predict(surge2_x_test)

Result:

## Generate output

In [82]:
# transform clf output
surge_pred = np.concatenate((inverse_transform(surge1_y_test), inverse_transform(surge2_y_test)), axis=1)

In [85]:
y_columns = [f'surge1_t{i}' for i in range(10)] + [f'surge2_t{i}' for i in range(10)]
Y_pred = pd.DataFrame(data=surge_pred, columns=y_columns, index=X_test['id_sequence'])
Y_pred.to_csv('Y_sep_MLP.csv', index_label='id_sequence', sep=',')